In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt


In [2]:
IMG_SIZE = 224
batch_size = 64

_, breed_info = tfds.load(
    'stanford_dogs', with_info=True, as_supervised=True
)

_, cats_dogs_info = tfds.load(
    'cats_vs_dogs', with_info=True, as_supervised=True
)


print(f'cats_vs_dogs: {cats_dogs_info.features["label"].num_classes}')
print(f'dog breeds: {breed_info.features["label"].num_classes}')


cats_vs_dogs: 2
dog breeds: 120


In [12]:
# print(breed_info.features["label"].num_classes)
# print(breed_info.features["label"].names)
with open('breeds.txt', 'w') as f:
    for name in breed_info.features['label'].names:
        f.write(''.join(name.split('-')[1:]) + '\n')

# print(breed_info.features["label"].int2str(7))  # Human readable version (8 -> 'cat')
# print(breed_info.features["label"].str2int('7'))

In [14]:
def get_breeds(file):
    breeds = []
    with open(file, 'r') as f:
        for line in f:
            breeds.append(line.strip())
    return breeds

print(get_breeds('breeds.txt'))

['chihuahua', 'japanese_spaniel', 'maltese_dog', 'pekinese', 'shihtzu', 'blenheim_spaniel', 'papillon', 'toy_terrier', 'rhodesian_ridgeback', 'afghan_hound', 'basset', 'beagle', 'bloodhound', 'bluetick', 'blackandtan_coonhound', 'walker_hound', 'english_foxhound', 'redbone', 'borzoi', 'irish_wolfhound', 'italian_greyhound', 'whippet', 'ibizan_hound', 'norwegian_elkhound', 'otterhound', 'saluki', 'scottish_deerhound', 'weimaraner', 'staffordshire_bullterrier', 'american_staffordshire_terrier', 'bedlington_terrier', 'border_terrier', 'kerry_blue_terrier', 'irish_terrier', 'norfolk_terrier', 'norwich_terrier', 'yorkshire_terrier', 'wirehaired_fox_terrier', 'lakeland_terrier', 'sealyham_terrier', 'airedale', 'cairn', 'australian_terrier', 'dandie_dinmont', 'boston_bull', 'miniature_schnauzer', 'giant_schnauzer', 'standard_schnauzer', 'scotch_terrier', 'tibetan_terrier', 'silky_terrier', 'softcoated_wheaten_terrier', 'west_highland_white_terrier', 'lhasa', 'flatcoated_retriever', 'curlycoat

In [ ]:
# build model
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

img_augmentation = Sequential(
    [
        preprocessing.RandomRotation(factor=0.15),
        preprocessing.RandomTranslation(height_factor=0.1, width_factor=0.1),
        preprocessing.RandomFlip(),
        preprocessing.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)

def build_model(num_classes):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = img_augmentation(inputs)
    model = EfficientNetB0(include_top=False, input_tensor=x, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(2, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model
cats_dogs_model = build_model(num_classes=2)

In [ ]:
tf.__version__


In [5]:
dogs_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)

# cats_dogs_model.load_weights('../tf_models/colab/cats_dogs_best_colab1.h5')

dogs_model = tf.keras.models.load_model('best_local1.h5', compile=False)
dogs_model.compile(
        optimizer=dogs_optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )

In [ ]:
dogs_model.summary()

In [ ]:
from utils import predict_image

file = 'labrador.jpg'

predict_image(file, dogs_model, breed_info)


In [ ]:
predict_image(file, cats_dogs_model, cats_dogs_info)